In [7]:
import requests
import time
from pprint import pprint


ip='45.113.232.43'
# ip='localhost'
req_body = {
    "domain":"(define (domain blocksworld) (:requirements :strips) (:predicates (on ?x ?y) (ontable ?x) (clear ?x) (arm-free) (holding ?x) ) (:action pick-up :parameters (?x) :precondition (and (clear ?x) (ontable ?x) (arm-free)) :effect (and (not (ontable ?x)) (not (clear ?x)) (not (arm-free)) (holding ?x))) (:action put-down :parameters (?x) :precondition (holding ?x) :effect (and (not (holding ?x)) (clear ?x) (arm-free) (ontable ?x))) (:action stack :parameters (?x ?y) :precondition (and (holding ?x) (clear ?y)) :effect (and (not (holding ?x)) (not (clear ?y)) (clear ?x) (arm-free) (on ?x ?y))) (:action unstack :parameters (?x ?y) :precondition (and (on ?x ?y) (clear ?x) (arm-free)) :effect (and (holding ?x) (clear ?y) (not (clear ?x)) (not (arm-free)) (not (on ?x ?y)))))",
    "problem":"(define (problem blocksworld-4-2) (:domain blocksworld) (:objects b d c a ) (:init (clear a) (clear c) (clear d) (ontable a) (ontable b) (ontable d) (on c b) (arm-free)) (:goal (and (on a b) (on b c) (on c d))))"
}

In [23]:
def test_planner(package_name,req_body,service="solve"):
    service_url="http://"+ip+":5001/package/"+package_name+"/"+service
    solve_request = requests.post(service_url, json=req_body).json()
    print(solve_request)
    celery_result = requests.get("http://"+ip+":5001" + solve_request['result'])
    print('Computing...')
    while celery_result.text == 'PENDING':
        time.sleep(12)
        celery_result = requests.get("http://"+ip+":5001" + solve_request['result'])
        
    ttt=celery_result
    pprint(celery_result.json())

In [24]:
test_planner("dual-bfws-fdparser",req_body)

{'result': '/check/d009e66a-ad23-423a-9c8b-9efd5355ce55?external=True'}
Computing...
{'result': {'call': 'dual-bfws-fdparser domain problem plan',
            'output': {'plan': '(unstack c b)\n'
                               '(stack c d)\n'
                               '(pick-up b)\n'
                               '(stack b c)\n'
                               '(pick-up a)\n'
                               '(stack a b)\n'},
            'stderr': '',
            'stdout': 'Domain: domain Problem: problem\n'
                      'Parsing...\n'
                      'Parsing: [0.010s CPU, 0.001s wall-clock]\n'
                      'Generating Datalog program... [0.000s CPU, 0.000s '
                      'wall-clock]\n'
                      'Normalizing Datalog program...\n'
                      'Normalizing Datalog program: [0.000s CPU, 0.003s '
                      'wall-clock]\n'
                      'Preparing model... [0.000s CPU, 0.000s wall-clock]\n'
                    

In [25]:
test_planner("dual-bfws-ffparser",req_body)

{'result': '/check/aec398a9-4aa4-4605-8736-1b0397e61720?external=True'}
Computing...
{'result': {'call': 'dual-bfws-ffparser domain problem plan',
            'output': {'plan': '(UNSTACK C B)\n'
                               '(STACK C D)\n'
                               '(PICK-UP B)\n'
                               '(STACK B C)\n'
                               '(PICK-UP A)\n'
                               '(STACK A B)\n'},
            'stderr': '',
            'stdout': ' --- OK.\n'
                      ' Match tree built with 40 nodes.\n'
                      '\n'
                      'PDDL problem description loaded: \n'
                      '\tDomain: BLOCKSWORLD\n'
                      '\tProblem: BLOCKSWORLD-4-2\n'
                      '\t#Actions: 40\n'
                      '\t#Fluents: 29\n'
                      'Goals found: 3\n'
                      'Goals_Edges found: 3\n'
                      'Starting search with 1-BFWS...\n'
                      '--[3 / 0]

In [32]:
# lama
# test_planner("lama",req_body)

In [31]:
# kstar
# test_planner("kstar",req_body)

In [26]:
# lama-first
# test_planner("lama-first",req_body)

In [27]:
# cerberus-sat
# test_planner("cerberus-sat",req_body)

In [28]:
# enhsp-2020
# test_planner("enhsp-2020",req_body)

In [29]:
# forbiditerative-topk 1 plan
# topk_body=req_body.copy()
# topk_body["number_of_plans"]=1
# test_planner("forbiditerative-topk",topk_body)

In [30]:
# forbiditerative-topk 3 plan
# topk_body=req_body.copy()
# topk_body["number_of_plans"]=3
# test_planner("forbiditerative-topk",topk_body)